# Inference

In [1]:
# imports
import ast
import os
import pathlib

import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn_tutorial.backbone_resnet import ResNetBackbones
from pytorch_faster_rcnn_tutorial.datasets import (
    ObjectDetectionDataSet,
    ObjectDetectionDatasetSingle,
)
from pytorch_faster_rcnn_tutorial.faster_RCNN import get_faster_rcnn_resnet
from pytorch_faster_rcnn_tutorial.transformations import (
    ComposeDouble,
    ComposeSingle,
    FunctionWrapperDouble,
    FunctionWrapperSingle,
    apply_nms,
    apply_score_threshold,
    normalize_01,
)
from pytorch_faster_rcnn_tutorial.utils import (
    collate_single,
    get_filenames_of_path,
    save_json,
)
from pytorch_faster_rcnn_tutorial.viewers.object_detection_viewer import (
    ObjectDetectionViewer,
    ObjectDetectionViewerSingle,
)
from training_script import NeptuneSettings


In [2]:
# parameters
params = {
    "EXPERIMENT": "GMT3-357",  # experiment name, e.g. Head-42
    "OWNER": "mikmikthemimic",  # e.g. johndoe55
    "INPUT_DIR": "pytorch_faster_rcnn_tutorial/data/heads/test",  # files to predict
    "PREDICTIONS_PATH": "predictions",  # where to save the predictions
    "MODEL_DIR": "model/kfolds/best_model.pt",  # load model from checkpoint
    "DOWNLOAD": True,  # whether to download from neptune
    "DOWNLOAD_PATH": "model",  # where to save the model if DOWNLOAD is True
    "PROJECT": "GM-Thesis3",  # Project name

    "ENSEMBLE": False,  # nilagay ko lang to for the ensemble model
}


In [3]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params["INPUT_DIR"]))
inputs.sort()
print(f"Found {len(inputs)} files in {params['INPUT_DIR']}")

Found 640 files in pytorch_faster_rcnn_tutorial/data/heads/test


In [4]:
# transformations
transforms = ComposeSingle(
    [
        FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
        FunctionWrapperSingle(normalize_01),
    ]
)


In [5]:
# create dataset
dataset = ObjectDetectionDatasetSingle(
    inputs=inputs,
    transform=transforms,
    use_cache=False,
)


In [6]:
# create dataloader
dataloader_prediction = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_single,
)


In [7]:
# environment variables (pydantic BaseSettings class)
neptune_settings: NeptuneSettings = NeptuneSettings()


In [8]:
# import experiment from neptune
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
print(f"Project: {project_name}")
project = neptune.init_run(
    project=project_name,
    api_token=neptune_settings.api_key,
    with_id=params["EXPERIMENT"],
    mode="read-only",
)  # get project
experiment_id = params["EXPERIMENT"]  # experiment id
parameters = project['training/hyperparams'].fetch()


Project: mikmikthemimic/GM-Thesis3
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mikmikthemimic/GM-Thesis3/e/GMT3-357


In [9]:
print(project['training/hyperparams'].fetch())

{'ACCELERATOR': 'cuda', 'ANCHOR_SIZE': '((32,), (64,), (128,), (256,))', 'ASPECT_RATIOS': '((0.5, 1.0, 2.0),)', 'BACKBONE': 'ResNetBackbones.RESNET34', 'BATCH_SIZE': 6, 'CACHE': True, 'CLASSES': 4, 'FAST_DEV_RUN': False, 'FOLDS': 5, 'FPN': True, 'IMG_MEAN': '[0.485, 0.456, 0.406]', 'IMG_STD': '[0.229, 0.224, 0.225]', 'IOU_THRESHOLD': 0.6, 'LOG_MODEL': True, 'LR': 0.002, 'MAXEPOCHS': 20, 'MAX_SIZE': 1025, 'MIN_SIZE': 1024, 'PATIENCE': 50, 'PRECISION': 32, 'SAVE_DIR': '/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial', 'SEED': 42, 'iou_threshold': 0.6, 'lr': 0.002, 'model': 'None'}


In [10]:
# rcnn transform
transform = GeneralizedRCNNTransform(
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
    image_mean=ast.literal_eval(parameters["IMG_MEAN"]),
    image_std=ast.literal_eval(parameters["IMG_STD"]),
    box_nms_thresh = 0.6
)


In [11]:
# color mapping
color_mapping = {
    1: "blue",
    2: "green",
    3: "white",
    4: "yellow",
    5: "red"
}


In [12]:
# download model from neptune or load from checkpoint
if params["DOWNLOAD"]:
    download_path = pathlib.Path(os.getcwd()) / params["DOWNLOAD_PATH"]
    download_path.mkdir(parents=True, exist_ok=True)
    if params["ENSEMBLE"]:
        model_name = "ensemble_model.pt"  # that's how I called the best model
        # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
        if not (download_path / model_name).is_file():
            project['artifacts/ensemble_model'].download(
                destination=download_path.as_posix()
            )  # download model

        model_state_dict = torch.load(
            download_path / model_name, map_location=torch.device("cpu")
        )
    else:
        model_name = "best_model.pt"  # that's how I called the best model
        # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
        if not (download_path / model_name).is_file():
            project['artifacts/best_model'].download(
                destination=download_path.as_posix()
            )  # download model

        model_state_dict = torch.load(
            download_path / model_name, map_location=torch.device("cpu")
        )
else:
    checkpoint = torch.load(params["MODEL_DIR"], map_location=torch.device("cpu"))
    model_state_dict = checkpoint["hyper_parameters"]["model"].state_dict()


In [13]:
model = get_faster_rcnn_resnet(
    num_classes=int(parameters["CLASSES"]),
    backbone_name=ResNetBackbones(parameters["BACKBONE"].split(".")[1].lower()),  # reverse look-up enum
    anchor_size=ast.literal_eval(parameters["ANCHOR_SIZE"]),
    aspect_ratios=ast.literal_eval(parameters["ASPECT_RATIOS"]),
    fpn=ast.literal_eval(str(parameters['FPN'])),
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
)


C:\Users\abiga\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abiga\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
# Check if keys start with model. and remove it
model_state_dict = {k[6:]: v for k, v in model_state_dict.items() if k.startswith("model.")}

In [15]:
# load weights
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [16]:
# inference (cpu)
model.eval()
for sample in dataloader_prediction:
    x, x_name = sample
    with torch.no_grad():
        pred = model(x)
        pred = {key: value.numpy() for key, value in pred[0].items()}
        name = pathlib.Path(x_name[0])
        save_dir = pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
        save_dir.mkdir(parents=True, exist_ok=True)
        pred_list = {
            key: value.tolist() for key, value in pred.items()
        }  # numpy arrays are not serializable -> .tolist()
        save_json(pred_list, path=save_dir / name.with_suffix(".json"))


In [29]:
# get prediction files
predictions = get_filenames_of_path(
    pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
)
predictions.sort()

# create prediction dataset
iou_threshold = 0.5
score_threshold = 0.7

transforms_prediction = ComposeDouble(
    [
        FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01),
        FunctionWrapperDouble(
            apply_nms, input=False, target=True, iou_threshold=iou_threshold
        ),
        FunctionWrapperDouble(
            apply_score_threshold,
            input=False,
            target=True,
            score_threshold=score_threshold,
        ),
    ]
)

dataset_prediction = ObjectDetectionDataSet(
    inputs=inputs, targets=predictions, transform=transforms_prediction, use_cache=False, convert_to_format="xywh"
)

color_mapping = {
    1: "blue",
    2: "green",
}

In [30]:
# visualize predictions
datasetviewer_prediction = ObjectDetectionViewer(
    dataset=dataset_prediction, color_mapping=color_mapping
)

In [31]:
print(len(dataset_prediction))

640
